In [1]:
import pandas as pd

In [ ]:
# 全画像のラベルを csv から取得
df = pd.read_csv('labels.csv')
len(df)

In [ ]:
# ランダムに5件のデータを表示
df.sample(5)

In [ ]:
# ラベルの分布を確認
df['label'].value_counts()

In [5]:
import cohere
import os
from dotenv import load_dotenv, find_dotenv
from PIL import Image
from io import BytesIO
import base64
from tqdm import tqdm

In [6]:
_= load_dotenv(find_dotenv())

In [ ]:
api_key = os.getenv("COHERE_API_KEY")
model_id = os.getenv("COHERE_EMBED_MODEL_ID")
model_id

In [8]:
# 画像を base64 に変換してデータ URL を返す関数
def image_to_base64_data_url(image_path):
    with Image.open(image_path) as img:
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")
    data_url = f"data:image/jpeg;base64,{img_base64}"
    return data_url

In [9]:
co = cohere.Client(api_key=api_key)

In [ ]:
# CSVファイルのヘッダーを初期化（最初の1回のみ）
pd.DataFrame(columns=['image_path', 'label', 'embedding']).to_csv('image_embeddings.csv', index=False)

# DataFrameをイテレーション
for index, row in tqdm(df.iterrows(), total=len(df)):
    image_path = row['image_path']
    label = row['label']
    
    data_url = image_to_base64_data_url(image_path)
    ret = co.embed(
        input_type="image",
        images=[data_url],
        model=model_id,
        embedding_types=["float"],                
    )
    
    # 1件分のデータをDataFrameとして作成
    embedding_df = pd.DataFrame([{
        'image_path': image_path,
        'label': label,
        'embedding': ret.embeddings.float[0]
    }])
    
    # mode='a'（append）とheader=Falseで追記モードで保存
    embedding_df.to_csv('image_embeddings.csv', mode='a', header=False, index=False)